# Generación de texto creativo con IA - Poesía

### Librerías

In [ ]:
from transformers import XGLMTokenizer, XGLMForCausalLM
from transformers import DataCollatorWithPadding, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from datasets import load_metric
from datasets import load_dataset
from tokenizers import AddedToken
import pandas as pd
import gradio as gr


### Carga de modelo, tokenizador y ajuste de tokenizador

In [ ]:
tokenizer = XGLMTokenizer.from_pretrained("facebook/xglm-564M")
model = XGLMForCausalLM.from_pretrained("facebook/xglm-564M", device_map="cuda", cache_dir= 'I:/transformers')

In [ ]:
special_tokens = {
    'additional_special_tokens': [AddedToken('\n')]
}

# Modifica los tokens especiales del tokenizador
tokenizer.add_special_tokens(special_tokens)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

### Adquisición de conjunto de datos de poesía, limpieza y pre-procesado

In [ ]:
# Cargar el conjunto de datos
dataset = load_dataset("linhd-postdata/poesias")

# Acceder a la información del conjunto de datos
print(dataset)

In [ ]:
dataset_spanish = dataset.filter(lambda example: example['language'] == 'es')

In [ ]:
def format_ds(example):
  example["text"] = "<s>" + example['text'] + "</s>"
  return example

In [ ]:
dataset_spanish = dataset_spanish.map(format_ds)

In [ ]:
tokenized_data = dataset_spanish.map(
    lambda example: tokenizer(example["text"], max_length=512, truncation=True),
    batched=True,
)


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,  mlm=False)

### Entrenamiento y evaluación por época

In [ ]:
training_args = TrainingArguments(
    output_dir='I:/transformers/xglmsmall3',
    learning_rate= 0.0001, #2e-5,
    save_steps = 1000000,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.0001,
    gradient_accumulation_steps=2,
    do_eval = True,
    evaluation_strategy  = 'epoch',
    eval_steps = 1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

#### Guardado del modelo y métricas de rendimiento

In [ ]:

model.save_pretrained("C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/modelos_generadores/xglm_512m_poesia")

In [ ]:
logh = trainer.state.log_history

In [ ]:
extracted_data = []

# Itera sobre cada diccionario en la lista 'data'
for entry in logh:
    # Extrae las claves 'loss', 'step' y 'eval_loss' de cada diccionario
    extracted_entry = {'loss': entry.get('loss', None),
                       'step': entry.get('step', None),
                       'eval_loss': entry.get('eval_loss', None)}

    # Agrega el diccionario extraído a la nueva lista
    extracted_data.append(extracted_entry)



In [ ]:
# Crea un DataFrame de pandas con los datos extraídos
df = pd.DataFrame(extracted_data)
csv_file_path = 'C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/metricas_modelo/loss_xglm_512m.csv'
df.to_csv(csv_file_path, index=False)

### Despliegue

#### Carga de modelo, tokenizador y ajuste de tokenizador

In [ ]:
ruta_modelo = "C:/Users/Victor/OneDrive/Documentos/MasterIA/09_TFM/desarrollo/modelosfinal/modelos_generadores/xglm_512m_poesia"
# XGLMTokenizerFast, XGLMForCausalLM
# Cargar el tokenizador y el modelo
model = XGLMForCausalLM.from_pretrained(ruta_modelo, device_map="cuda")
tokenizer = XGLMTokenizerFast.from_pretrained("facebook/xglm-564M")

In [ ]:
special_tokens = {
    'additional_special_tokens': [AddedToken('\n')]
}

# Modifica los tokens especiales del tokenizador
tokenizer.add_special_tokens(special_tokens)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

#### Función generadora de poesía

In [ ]:
def generar_texto(Texto):
    input_ids = tokenizer.encode(Texto, return_tensors="pt").to('cuda')

    # Generar texto condicionalmente
    output = model.generate(input_ids, max_length=250, min_length=100,pad_token_id=tokenizer.eos_token_id,
                        #repetition_penalty = 1.2, num_beams=1, 
                        #num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, 
                        temperature=0.95, do_sample=True,)

# Decodificar y mostrar el texto generado
    texto_generado = tokenizer.decode(output[0], skip_special_tokens=False)

    
    #texto_generado = "Texto generado por el modelo..." # Aquí deberías llamar a tu modelo generativo para generar texto
    
    return texto_generado

#### Interfaz web para generar poesía con Gradio

In [ ]:
interfaz = gr.Interface(
    fn=generar_texto,
    inputs="text",
    outputs="text",
    title="Generador de Poesía",
    description="Introduce un texto inicial y genera poesía",
    allow_flagging = 'never',
    clear_btn = 'Nueva Poesía',
    submit_btn = 'Crea Poesía'

)

In [ ]:
interfaz.launch(share=True)

In [ ]:
interfaz.close()